In [1]:
!pip install scikit-optimize
!pip install pyriemann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyriemann: filename=pyriemann-0.5-py2.py3-none-any.whl size=107753 sha256=5f4d3ba359bc995f3dfb0fcf2f139f1246928015914f80157cd503086549e40e
  Stored in directory: /root/.cache/pip/wheels/84/86/79/622e9c1dc933dc088e287ebfaac5aa9bdc6a38a9db193ce1f1
Successfully built pyriemann


In [2]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import utils as np_utils
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.optimizers.experimental import AdamW
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import math
import gc
from numpy import mean
from numpy import std
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score
from skopt.utils import use_named_args
from skopt import gp_minimize
import tensorflow as tf

In [3]:
'''
  input1   = Input(shape = (1, Chans, Samples))

  ##################################################################
  block1       = Conv2D(F1, (1, kernelLength), padding = 'same',
                                  input_shape = (1, Chans, Samples),
                                  use_bias = False)(input1)
  block1       = BatchNormalization(axis = 1)(block1)
  block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                  depth_multiplier = D,
                                  depthwise_constraint = max_norm(norm_rate))(block1)
  block1       = BatchNormalization(axis = 1)(block1)
  block1       = Activation('elu')(block1)
  block1       = AveragePooling2D((1, 4))(block1)
  block1       = dropoutType(dropoutRate)(block1)

  block2       = SeparableConv2D(F2, (1, 16),
                                  use_bias = False, padding = 'same')(block1)
  block2       = BatchNormalization(axis = 1)(block2)
  block2       = Activation('elu')(block2)
  block2       = AveragePooling2D((1, 8))(block2)
  block2       = dropoutType(dropoutRate)(block2)

  flatten      = Flatten(name = 'flatten')(block2)

  dense        = Dense(nb_classes, name = 'dense',
                        kernel_constraint = max_norm(norm_rate))(flatten)
  softmax      = Activation('softmax', name = 'softmax')(dense)

  model        = Model(inputs=input1, outputs=softmax)

  if optimizer_type == 'Adam':
    optimizer = Adam(learning_rate = lr)
  if optimizer_type == 'Adamax':
    optimizer = Adamax(learning_rate = lr)
  if optimizer_type == 'AdamW':
    optimizer = AdamW(learning_rate = lr)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['accuracy'])
  return model
'''

"\n  input1   = Input(shape = (1, Chans, Samples))\n\n  ##################################################################\n  block1       = Conv2D(F1, (1, kernelLength), padding = 'same',\n                                  input_shape = (1, Chans, Samples),\n                                  use_bias = False)(input1)\n  block1       = BatchNormalization(axis = 1)(block1)\n  block1       = DepthwiseConv2D((Chans, 1), use_bias = False, \n                                  depth_multiplier = D,\n                                  depthwise_constraint = max_norm(norm_rate))(block1)\n  block1       = BatchNormalization(axis = 1)(block1)\n  block1       = Activation('elu')(block1)\n  block1       = AveragePooling2D((1, 4))(block1)\n  block1       = dropoutType(dropoutRate)(block1)\n  \n  block2       = SeparableConv2D(F2, (1, 16),\n                                  use_bias = False, padding = 'same')(block1)\n  block2       = BatchNormalization(axis = 1)(block2)\n  block2       = Activation('

In [4]:
'''
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')
'''

"\nfrom tensorflow.keras import backend as K\nK.set_image_data_format('channels_first')\n"

In [5]:
'''
input1   = Input(shape = (8, 7, 1))
block1       = Conv2D(16, (1, 3), padding = 'same',
                                input_shape = (1, 8, 7),
                                use_bias = False)(input1)
block1       = BatchNormalization(axis = 1)(block1)
block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                  depth_multiplier = 2,
                                  depthwise_constraint = max_norm(0.5))(block1)
block3       = MaxPooling2D((1, 2))(block2)

model        = Model(inputs=input1, outputs=block3)
'''

"\ninput1   = Input(shape = (8, 7, 1))\nblock1       = Conv2D(16, (1, 3), padding = 'same',\n                                input_shape = (1, 8, 7),\n                                use_bias = False)(input1)\nblock1       = BatchNormalization(axis = 1)(block1)\nblock2       = DepthwiseConv2D((8, 1), use_bias = False, \n                                  depth_multiplier = 2,\n                                  depthwise_constraint = max_norm(0.5))(block1)\nblock3       = MaxPooling2D((1, 2))(block2)\n\nmodel        = Model(inputs=input1, outputs=block3)\n"

In [11]:
'''
##########Model 3
input1       = Input(shape = (8, 7, 1))
block1       = Conv2D(16, (1, 3), padding = 'same',
                                input_shape = (1, 8, 7),
                                use_bias = False)(input1)
block1       = BatchNormalization(axis = -1)(block1)
block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                  depth_multiplier = 2,
                                  depthwise_constraint = max_norm(0.5))(block1)
block2       = BatchNormalization(axis = -1)(block2)
block2       = Activation('relu')(block2)
block3       = Dropout(0.2)(block2)
block4       = SeparableConv2D(32, (1, 2),
                                use_bias = False, padding = 'same')(block3)
block5       = BatchNormalization(axis = -1)(block4)
block6       = Activation('relu')(block5)
flatten      = Flatten(name = 'flatten')(block6)
block7       = Dropout(0.2)(flatten)
dense1       = Dense(10, name = 'dense1')(block7)
dense2       = Dense(2, name = 'dense2',
                      kernel_constraint = max_norm(0.5))(dense1)
softmax      = Activation('softmax', name = 'softmax')(dense2)

model        = Model(inputs=input1, outputs=softmax)
'''

In [21]:
##########Model 4
input1       = Input(shape = (8, 7, 1))
block1       = Conv2D(16, (1, 5), padding = 'same',
                                input_shape = (1, 8, 7),
                                use_bias = False)(input1)
block1       = BatchNormalization(axis = -1)(block1)
block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                  depth_multiplier = 4,
                                  depthwise_constraint = max_norm(0.5))(block1)
block2       = BatchNormalization(axis = -1)(block2)
block2       = Activation('relu')(block2)
block3       = Dropout(0.2)(block2)
block4       = SeparableConv2D(32, (1, 2),
                                use_bias = False, padding = 'same')(block3)
block5       = BatchNormalization(axis = -1)(block4)
block6       = Activation('relu')(block5)
block6       = MaxPooling2D((1, 2))(block6)
flatten      = Flatten(name = 'flatten')(block6)
block7       = Dropout(0.2)(flatten)
dense1       = Dense(10, name = 'dense1')(block7)
dense2       = Dense(2, name = 'dense2',
                      kernel_constraint = max_norm(0.5))(dense1)
softmax      = Activation('softmax', name = 'softmax')(dense2)

model        = Model(inputs=input1, outputs=softmax)


In [ ]:
''''
#Model 2
input1       = Input(shape = (8, 7, 1))
block1       = Conv2D(16, (1, 3), padding = 'same',
                                input_shape = (1, 8, 7),
                                use_bias = False)(input1)
block1       = BatchNormalization(axis = -1)(block1)
block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                  depth_multiplier = 2,
                                  depthwise_constraint = max_norm(0.5))(block1)
block2       = BatchNormalization(axis = -1)(block2)
block2       = Activation('relu')(block2)
block3       = MaxPooling2D((1, 2))(block2)
flatten      = Flatten(name = 'flatten')(block3)
block4       = Dropout(0.2)(flatten)
dense        = Dense(2, name = 'dense',
                      kernel_constraint = max_norm(0.5))(block4)
softmax      = Activation('softmax', name = 'softmax')(dense)

model        = Model(inputs=input1, outputs=softmax)


In [ ]:
'''
#Model 1
input1       = Input(shape = (8, 7, 1))
block1       = Conv2D(16, (1, 3), padding = 'same',
                                input_shape = (1, 8, 7),
                                use_bias = False)(input1)
block1       = BatchNormalization(axis = -1)(block1)
block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                  depth_multiplier = 2,
                                  depthwise_constraint = max_norm(0.5))(block1)
block2       = BatchNormalization(axis = -1)(block2)
block2       = Activation('relu')(block2)
flatten      = Flatten(name = 'flatten')(block2)
block3       = Dropout(0.2)(flatten)
dense        = Dense(2, name = 'dense',
                      kernel_constraint = max_norm(0.5))(block3)
softmax      = Activation('softmax', name = 'softmax')(dense)

model        = Model(inputs=input1, outputs=softmax)
'''

In [ ]:
def create_model1(optimizer='adam', kernel_size=3, dropout=0.5, norm_rate=0.01, F1=16, kernelsize=3, D=2):

    input1       = Input(shape = (8, 7, 1))
    block1       = Conv2D(F1, (1, kernelsize), padding = 'same',
                                    input_shape = (1, 8, 7),
                                    use_bias = False)(input1)
    block1       = BatchNormalization(axis = -1)(block1)
    block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                      depth_multiplier = D,
                                      depthwise_constraint = max_norm(norm_rate))(block1)
    block2       = BatchNormalization(axis = -1)(block2)
    block2       = Activation('relu')(block2)
    flatten      = Flatten(name = 'flatten')(block2)
    block3       = Dropout(dropout)(flatten)
    dense        = Dense(2, name = 'dense',
                          kernel_constraint = max_norm(norm_rate))(block3)
    softmax      = Activation('softmax', name = 'softmax')(dense)

    model        = Model(inputs=input1, outputs=softmax)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [22]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 8, 7, 1)]         0         
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 7, 16)          80        
                                                                 
 batch_normalization_22 (Bat  (None, 8, 7, 16)         64        
 chNormalization)                                                
                                                                 
 depthwise_conv2d_8 (Depthwi  (None, 1, 7, 64)         512       
 seConv2D)                                                       
                                                                 
 batch_normalization_23 (Bat  (None, 1, 7, 64)         256       
 chNormalization)                                                
                                                           